In [1]:
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
day = 10

In [2]:
feat_list = [
            'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover'
#             "open_ts_norm_30d","high_ts_norm_30d","low_ts_norm_30d","close_ts_norm_30d","vwap_ts_norm_30d" 
#             ,"volume_ts_norm_30d" ,"pct_chg_ts_norm_30d" ,"turnover_ts_norm_30d" ,"free_turnover_ts_norm_30d" 
]

In [3]:
dict_load = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/base_list.npy",allow_pickle=True)
dict_load = dict_load.item()
dict_load

{'open': 'open',
 'high': 'high',
 'low': 'low',
 'close': 'close',
 'vwap': 'vwap',
 'volume': 'volume',
 'pct_chg': 'pct_chg',
 'turnover': 'turnover',
 'free_turnover': 'free_turnover',
 'open_ts_norm_30d': 'div(sub(open,mavg(30,open)),mstd(30,open))',
 'high_ts_norm_30d': 'div(sub(high,mavg(30,high)),mstd(30,high))',
 'low_ts_norm_30d': 'div(sub(low,mavg(30,low)),mstd(30,low))',
 'close_ts_norm_30d': 'div(sub(close,mavg(30,close)),mstd(30,close))',
 'vwap_ts_norm_30d': 'div(sub(vwap,mavg(30,vwap)),mstd(30,vwap))',
 'volume_ts_norm_30d': 'div(sub(volume,mavg(30,volume)),mstd(30,volume))',
 'pct_chg_ts_norm_30d': 'div(sub(pct_chg,mavg(30,pct_chg)),mstd(30,pct_chg))',
 'turnover_ts_norm_30d': 'div(sub(turnover,mavg(30,turnover)),mstd(30,turnover))',
 'free_turnover_ts_norm_30d': 'div(sub(free_turnover,mavg(30,free_turnover)),mstd(30,free_turnover))',
 'open_residual': 'sub(log(open),log(delay(1,open)))',
 'high_residual': 'sub(log(high),log(delay(1,high)))',
 'low_residual': 'sub(log(

In [8]:
name_list = []
task_list = []
for i in name_list2:
    name_list.append(i + "_TsNorm30d")
#     print(alpha_list[i])
    task_list.append("div(sub(%s,mavg(30,%s)),mstd(30,%s))" % (alpha_list[i],alpha_list[i],alpha_list[i]))

In [4]:
# for i in feat_list:
#     dict_load["%s_residual" %i]  = "sub(log(%s),log(delay(1,%s)))" % (i,i)
# dict_load

In [30]:
# np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/base_list.npy",dict_load)
# np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/base_list.npy",dict_load)

In [34]:
# name_list = []
# task_list = []
# for i in dict_load.keys():
#     name_list.append(i)
#     task_list.append(dict_load[i])

In [4]:
name_list = []
task_list = []
# Moving Cov
for i in range(len(feat_list)-1):
    for j in range(i+1,len(feat_list)):
        name_list.append("COV_%s_%s_%s" % (feat_list[i],feat_list[j],day))
        task_list.append("mcovar(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# Moving Corr
for i in range(len(feat_list)):
    for j in range(i+1,len(feat_list)):
        name_list.append("CORR_%s_%s_%s" % (feat_list[i],feat_list[j],day))
        task_list.append("ts_corr(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# # Moving STD
for i in range(len(feat_list)):
    name_list.append("STD_%s_%s" % (feat_list[i],day))
    task_list.append("mstd(%s,%s)" % (day,dict_load[feat_list[i]]))
    
# # Z-score
for i in range(len(feat_list)):
    name_list.append("ZSCORE_%s_%s" % (feat_list[i],day)) 
    task_list.append("div(mavg(%s,%s),mstd(%s,%s))" % (day,dict_load[feat_list[i]],day,dict_load[feat_list[i]]))
    
# # Return
for i in range(len(feat_list)):
    name_list.append("RETURN_%s_%s" % (feat_list[i],day))
    task_list.append("div(sub(%s,delay(%s,%s)),delay(%s,%s))" % (dict_load[feat_list[i]],day,dict_load[feat_list[i]],day,dict_load[feat_list[i]])) 
# Decay 
for i in range(len(feat_list)):
    name_list.append("DECAY_%s_%s" % (feat_list[i],day))
    task_list.append("decay(%s,%s)" % (day,dict_load[feat_list[i]]))

In [38]:
final = ""
for i in name_list[80:]:
    final += i + ";"
print("name")
print(final)

name
STD_free_turnover_10_TsNorm30d;ZSCORE_open_10_TsNorm30d;ZSCORE_high_10_TsNorm30d;ZSCORE_low_10_TsNorm30d;ZSCORE_close_10_TsNorm30d;ZSCORE_vwap_10_TsNorm30d;ZSCORE_volume_10_TsNorm30d;ZSCORE_pct_chg_10_TsNorm30d;ZSCORE_turnover_10_TsNorm30d;ZSCORE_free_turnover_10_TsNorm30d;RETURN_open_10_TsNorm30d;RETURN_high_10_TsNorm30d;RETURN_low_10_TsNorm30d;RETURN_close_10_TsNorm30d;RETURN_vwap_10_TsNorm30d;RETURN_volume_10_TsNorm30d;RETURN_pct_chg_10_TsNorm30d;RETURN_turnover_10_TsNorm30d;RETURN_free_turnover_10_TsNorm30d;DECAY_open_10_TsNorm30d;DECAY_high_10_TsNorm30d;DECAY_low_10_TsNorm30d;DECAY_close_10_TsNorm30d;DECAY_vwap_10_TsNorm30d;DECAY_volume_10_TsNorm30d;DECAY_pct_chg_10_TsNorm30d;DECAY_turnover_10_TsNorm30d;DECAY_free_turnover_10_TsNorm30d;


In [37]:
final = ""
for i in task_list[80:]:
    final += i + ";"
print("task")
print(final)

task
div(sub(mstd(10,free_turnover),mavg(30,mstd(10,free_turnover))),mstd(30,mstd(10,free_turnover)));div(sub(div(mavg(10,open),mstd(10,open)),mavg(30,div(mavg(10,open),mstd(10,open)))),mstd(30,div(mavg(10,open),mstd(10,open))));div(sub(div(mavg(10,high),mstd(10,high)),mavg(30,div(mavg(10,high),mstd(10,high)))),mstd(30,div(mavg(10,high),mstd(10,high))));div(sub(div(mavg(10,low),mstd(10,low)),mavg(30,div(mavg(10,low),mstd(10,low)))),mstd(30,div(mavg(10,low),mstd(10,low))));div(sub(div(mavg(10,close),mstd(10,close)),mavg(30,div(mavg(10,close),mstd(10,close)))),mstd(30,div(mavg(10,close),mstd(10,close))));div(sub(div(mavg(10,vwap),mstd(10,vwap)),mavg(30,div(mavg(10,vwap),mstd(10,vwap)))),mstd(30,div(mavg(10,vwap),mstd(10,vwap))));div(sub(div(mavg(10,volume),mstd(10,volume)),mavg(30,div(mavg(10,volume),mstd(10,volume)))),mstd(30,div(mavg(10,volume),mstd(10,volume))));div(sub(div(mavg(10,pct_chg),mstd(10,pct_chg)),mavg(30,div(mavg(10,pct_chg),mstd(10,pct_chg)))),mstd(30,div(mavg(10,pct_chg)

In [10]:
alpha_list = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/feature_list.npy",allow_pickle=True)
alpha_list = alpha_list.item()
Cover = False
for i in range(len(task_list)):
    if name_list[i] in alpha_list and alpha_list[name_list[i]] != task_list[i] and Cover == False:
        raise ValueError("contain " + name_list[i])
    alpha_list[name_list[i]] = task_list[i]
check_list = list(os.listdir("/home/ShareFolder/feature_platform/ti0/wuwenjun"))
for i in alpha_list.keys():
    if i not in check_list:
        raise ValueError("No [%s] in feature_platform" %i)
alpha_list

{'COV_open_high_10': 'mcovar(10,open,high)',
 'COV_open_low_10': 'mcovar(10,open,low)',
 'COV_open_close_10': 'mcovar(10,open,close)',
 'COV_open_vwap_10': 'mcovar(10,open,vwap)',
 'COV_open_volume_10': 'mcovar(10,open,volume)',
 'COV_open_pct_chg_10': 'mcovar(10,open,pct_chg)',
 'COV_open_turnover_10': 'mcovar(10,open,turnover)',
 'COV_open_free_turnover_10': 'mcovar(10,open,free_turnover)',
 'COV_high_low_10': 'mcovar(10,high,low)',
 'COV_high_close_10': 'mcovar(10,high,close)',
 'COV_high_vwap_10': 'mcovar(10,high,vwap)',
 'COV_high_volume_10': 'mcovar(10,high,volume)',
 'COV_high_pct_chg_10': 'mcovar(10,high,pct_chg)',
 'COV_high_turnover_10': 'mcovar(10,high,turnover)',
 'COV_high_free_turnover_10': 'mcovar(10,high,free_turnover)',
 'COV_low_close_10': 'mcovar(10,low,close)',
 'COV_low_vwap_10': 'mcovar(10,low,vwap)',
 'COV_low_volume_10': 'mcovar(10,low,volume)',
 'COV_low_pct_chg_10': 'mcovar(10,low,pct_chg)',
 'COV_low_turnover_10': 'mcovar(10,low,turnover)',
 'COV_low_free_tur

In [11]:
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/feature_list.npy",alpha_list)
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/feature_list.npy",alpha_list)